# Softmax Regression 기본 개념 소개

여러개의 클래스가 있을때 그것을 예측하는 multinomial classification중에서도 가장많이 쓰는 softmax Regression 학습

클래스가 3개 이상일때에도 binary classification로 구분이 가능하다

A or not, B or not, C or not의 각각다른 binary classification으로도 구현가능

x - [ a ] - y햇 -> [w1x1+w2x2+w3x3]

x - [ b ] - y햇 -> [w1x1+w2x2+w3x3]

x - [ c ] - y햇 -> [w1x1+w2x2+w3x3]
(각각은독립) 을 각각 구현하면 복잡하고 어렵다.

 |
 
[ WA1 WA2 WA3 ] [ x1

  WB1 WB2 WB3     x2  

[ WC1 WC2 WC3 ]   x3 ]

= [ Y햇A Y햇b Y햇c ] (1x3)형태가 맞음

# Softmax classifier의 cost함수

y = [2.0, 1.0, 0.1] 라고 나온 데이터를 총합 1.0이되며 0~1사이의값을 가지는 값으로 바꾸고자한다.

->> [0.7, 0.2, 0.1] 확률로써 볼수있다.

one-hot encoding -> 하나만 1.0 나머지는 0.0으로 초기화

Cost function

cross-entropy

y햇(예측값)과 y(실제값)의 차이를 구하기위해 크로스엔트로피 함수를 사용

cross-entropy cost function

...?

Logistic cost VS cross entropy

두개의 cost function이 서로 같은데 왜 같은지?

In [1]:
import tensorflow as tf

In [2]:
x_data = [[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]]
## one-hot encoding 방식으로 표시를했음 2,2,2,1,1,1,0,0

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
## one-hot으로 표시할때는 y의 개수가 레이블의 갯수이다.
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))
            
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))
    
    all = sess.run(hypothesis, feed_dict={
                   X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))

0 2.6410284
200 0.6209566
400 0.5047993
600 0.42475915
800 0.35358664
1000 0.2818073
1200 0.2327321
1400 0.21134046
1600 0.19338754
1800 0.17812221
2000 0.16499776
[[8.8496767e-03 9.9114287e-01 7.4591067e-06]] [1]
[[8.8496767e-03 9.9114287e-01 7.4591067e-06]
 [8.3207041e-01 1.5918599e-01 8.7435171e-03]
 [2.0910655e-08 3.6294176e-04 9.9963701e-01]] [1 0 2]


# TensorFlow로 Fancy Softmax Classification 구현하기

softmax_cross_entropy_with_logits

logits = tf.matmul(X, W) + b
주어진 X에 대해서 학습할 W을 매트릭스곱을 한후 + bias

hypothesis = tf.nn.softmax(logits)
softmax머신에 통과시키면 확률로 계산이 되어서 나옴

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
Y > one hot으로 주어졌음

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)

### tf.one_hot and reshape

Y = tf.placeholder(tf.int32, [None, 1]) x의 수는 n개 므로 None, y는 1 # 0~6, shape=(?, 1)
                            
Y_one_hot = tf.one_hot(Y, nb_classes) y(0~6), 몇개의 클래스를 줄것인지(7) # one hot shape=(?,1,7)
 
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # shape=(?,7)

tf.reshape(텐서, [원하는 shape설정]) >> -1는 전체선택 

** onehot은 만약에 랭크가 N이면 output이 N+1을 준다.(한차원을 상승시킨다)
(차원을 상승시키는이유 : 0,3의 element가 이어져서 보여지기때문에 구분하기힘들어진다

[[0], [3]] =>  [[[1,0,0,0,0,0,0]], [[0,0,0,1,0,0,0]]]

In [1]:
##tf.reset_default_graph()

import tensorflow as tf
import numpy as np

In [2]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [3]:
nb_classes = 7

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])

In [5]:
Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [ ]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [ ]:
logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

In [ ]:
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)

In [ ]:
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [ ]:
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [23]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X:x_data, Y:y_data})
            print("Step: {:5}\tLoss: {:.3f}\tACC: {:.2%}".format(step, loss, acc))
    
    pred = sess.run(prediction, feed_dict={X:x_data})
    
    for p,y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Loss: 6.541	ACC: 3.96%
Step:   100	Loss: 0.720	ACC: 81.19%
Step:   200	Loss: 0.384	ACC: 90.10%
Step:   300	Loss: 0.277	ACC: 93.07%
Step:   400	Loss: 0.218	ACC: 94.06%
Step:   500	Loss: 0.180	ACC: 96.04%
Step:   600	Loss: 0.154	ACC: 98.02%
Step:   700	Loss: 0.134	ACC: 98.02%
Step:   800	Loss: 0.119	ACC: 99.01%
Step:   900	Loss: 0.107	ACC: 99.01%
Step:  1000	Loss: 0.097	ACC: 99.01%
Step:  1100	Loss: 0.089	ACC: 100.00%
Step:  1200	Loss: 0.082	ACC: 100.00%
Step:  1300	Loss: 0.076	ACC: 100.00%
Step:  1400	Loss: 0.072	ACC: 100.00%
Step:  1500	Loss: 0.067	ACC: 100.00%
Step:  1600	Loss: 0.063	ACC: 100.00%
Step:  1700	Loss: 0.060	ACC: 100.00%
Step:  1800	Loss: 0.057	ACC: 100.00%
Step:  1900	Loss: 0.054	ACC: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 Tru